In [1]:
!pip install mlflow boto3 awscli

In [2]:
!aws configure

^C


In [3]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-54-252-23-89.ap-southeast-2.compute.amazonaws.com:5000")

In [4]:
# Set or create an experiment
mlflow.set_experiment("Exp 4 - Handling Imbalanced Data")

<Experiment: artifact_location='s3://yt-chrome-plugin-bucket/854808903051526481', creation_time=1734179317927, experiment_id='854808903051526481', last_update_time=1734179317927, lifecycle_stage='active', name='Exp 4 - Handling Imbalanced Data', tags={}>

In [7]:
!pip install imbalanced-learn


   ---------------------------------------- 0.0/238.4 kB ? eta -:--:--
   --- ----------------------------------- 20.5/238.4 kB 640.0 kB/s eta 0:00:01
   --- ----------------------------------- 20.5/238.4 kB 640.0 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/238.4 kB 281.8 kB/s eta 0:00:01
   --------------- ----------------------- 92.2/238.4 kB 525.1 kB/s eta 0:00:01
   ---------------- --------------------- 102.4/238.4 kB 535.8 kB/s eta 0:00:01
   ----------------------------- -------- 184.3/238.4 kB 654.4 kB/s eta 0:00:01
   -------------------------------------- 238.4/238.4 kB 768.2 kB/s eta 0:00:00


In [8]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [11]:
import pandas as pd

# Use a raw string or double backslashes for the file path on Windows
file_path = r'E:\yt-chrome-plugin\chrome-plugin\data\processed\redit_preprocessing.csv'

# Load the CSV file and drop rows with NaN values in the 'clean_comment' column
df = pd.read_csv(file_path).dropna(subset=['clean_comment'])


In [12]:
# Step 1: Function to run the experiment
def run_imbalanced_experiment(imbalance_method):
    # Define parameters for the TF-IDF vectorizer
    ngram_range = (1, 3)
    max_features = 1000

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        df['clean_comment'], 
        df['category'], 
        test_size=0.2, 
        random_state=42, 
        stratify=df['category']  # Ensures class proportions are preserved
    )

    # Initialize the TF-IDF vectorizer with specified parameters
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
    
    # Transform the training and testing data into TF-IDF vectors
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)  # Use `transform` for test data

    # Handle class imbalance based on the specified method
    if imbalance_method == 'class_weight':
        class_weight = 'balanced'  # Assign balanced weights to address class imbalance
    else:
        class_weight = None

        # Apply oversampling using SMOTE
        if imbalance_method == 'oversampling':
            smote = SMOTE(random_state=42)
            X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)
        
        # Apply oversampling using ADASYN
        elif imbalance_method == 'adasyn':
            adasyn = ADASYN(random_state=42)
            X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)
        
        # Apply undersampling using RandomUnderSampler
        elif imbalance_method == 'undersampling':
            rus = RandomUnderSampler(random_state=42)
            X_train_vec, y_train = rus.fit_resample(X_train_vec, y_train)
        
        # Combine oversampling and undersampling using SMOTEENN
        elif imbalance_method == 'smote_enn':
            smote_enn = SMOTEENN(random_state=42)
            X_train_vec, y_train = smote_enn.fit_resample(X_train_vec, y_train)

    # Log the experiment with MLflow
    with mlflow.start_run() as run:
        # Set MLflow tags for experiment tracking
        mlflow.set_tag("mlflow.runName", f"Imbalance_{imbalance_method}_RandomForest_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "imbalance_handling")
        mlflow.set_tag("model_type", "RandomForestClassifier")
        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, imbalance handling method={imbalance_method}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Define RandomForestClassifier parameters
        n_estimators = 200
        max_depth = 15

        # Log model parameters
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("imbalance_method", imbalance_method)

        # Train the RandomForestClassifier
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
        model.fit(X_train_vec, y_train)

        # Make predictions on the test set
        y_pred = model.predict(X_test_vec)

        # Evaluate model performance
        accuracy = accuracy_score(y_pred, y_test)
        classification_rep = classification_report(y_test, y_pred, output_dict=True)

        # Log classification metrics to MLflow
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Generate and log the confusion matrix
        conf_matrix = confusion_matrix(y_pred, y_test)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, Imbalance={imbalance_method}")
        
        # Save the confusion matrix plot and log it as an artifact
        confusion_matrix_filename = f"confusion_matrix_{imbalance_method}.png"
        plt.savefig(confusion_matrix_filename)
        mlflow.log_artifact(confusion_matrix_filename)
        plt.close()

        # Log the trained model to MLflow
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_imbalance_{imbalance_method}")

# Step 7: Run experiments for different imbalance methods
imbalance_methods = ['class_weight', 'oversampling', 'adasyn', 'undersampling', 'smote_enn']

# Execute the experiment for each imbalance handling method
for method in imbalance_methods:
    run_imbalanced_experiment(method)


2025/01/23 13:44:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/01/23 13:44:38 INFO mlflow.tracking._tracking_service.client: 🏃 View run Imbalance_class_weight_RandomForest_TFIDF_Trigrams at: http://ec2-54-252-23-89.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/854808903051526481/runs/4c34cfe49a6c4e4ca0d01bd32535c907.
2025/01/23 13:44:38 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-54-252-23-89.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/854808903051526481.
2025/01/23 13:45:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/01/23 13:45:57 INFO mlflow.tracking._tracking_service.client: 🏃 View run Imbalance_oversampling_RandomForest_TFIDF_Trigrams at: http://ec2-54-2